In [1]:
import numpy as np
import matplotlib.pyplot as plt,math

In [2]:
def rgb2gray(I):
    Gray = 0.2989 * I[:, :, 0] + 0.5870 * I[:, :, 1] + 0.1140 * I[:, :, 2]
    return Gray

In [3]:
def image2vector(image):
    v = image.reshape(image.shape[0] * image.shape[1], 1)
    return v

In [5]:
def distance(x,y):
    d=(np.sum((x-y)**2))
    d=math.sqrt(d)
    return d

In [6]:
def histo(I):
    h=np.zeros(256)
    I=image2vector(I)
    I=np.round(I)
    for i in I:
          h[int(i)]=int(h[int(i)]+1)
    return h

In [63]:
def histo_cum(I):
    L=histo(I)
    for i in range(1,len(L)):
        L[i]=L[i]+L[i-1]
    return L
    

In [7]:
def avrage_color(I1):
    return [np.mean(I1)]

In [9]:
def variance(I):
    l,c=I.shape
    m=np.sum(I)/(l*c)
    v=np.sum((I-m)**2)/(l*c)
    return v

In [10]:
def energy(I):
    e=np.sum(I**2)
    return e


In [11]:
def contrast(I):
    l,c=I.shape;s=0
    for i in range(l):
        for j in range(c):
            s+=(i-j)**2*I[i,j]
    return s

In [12]:
def homogenite(I):
    L,C = I.shape
    mo=0
    for i in range(L) :
        for j in range(C) :
            mo+=I[i,j]/(1+np.abs(i-j))
    return mo

In [15]:
def entropie(I):
    nl,nc = np.shape(I)
    ent=0
    for i in range(nl):
        for j in range(nc):
            if I[i,j]!=0:
                ent+=I[i,j]*np.log2(I[i,j])
    return -ent

In [16]:
def stocker_images(nbr_im, filename):
    filename1=filename+'1.jpg'
    I=plt.imread(filename1)
    nl,nc=rgb2gray(I).shape
    mat_image=np.zeros((nl,nc,nbr_im))
    for i in range(nbr_im):
        filename1= filename+ str(i+1) + '.jpg'
        mat_image[:,:,i]=rgb2gray(plt.imread(filename1))       
    return mat_image

In [17]:
filename='image_database/'
nbr_im=66
mat_image=stocker_images(nbr_im, filename)

In [18]:
def texture(I):
    h=[variance(I),energy(I),entropie(I),contrast(I),homogenite(I)]
    return np.array(h)

In [192]:
def cooccurence(I):
    g=np.zeros((int(np.max(I)+1),int(np.max(I)+1)))
    l,c=np.shape(I)
    for i in range(l):
        for j in range(c-1):
            g[int(I[i,j]),int(I[i,j+1])]+=1
    return g

In [193]:
def texture_cooccurence(I):
    return texture(cooccurence(I))
    

In [179]:
def img_test(filename1):
    L=[]
    for i in range(0,6):
        for j in range (11*i+7,11*i+7+4):
            filename1=filename+str(j+1)+'.jpg'
            L.append(rgb2gray(plt.imread(filename1)))
    return np.array(L)

In [180]:
def img_train(filename1):
    L=[]
    for i in range(0,6):
        for j in range (11*i,11*i+7):
            filename1=filename+str(j+1)+'.jpg'
            L.append(rgb2gray(plt.imread(filename1)))
    return np.array(L)

In [181]:
mat_img_train=img_train(filename)
mat_img_test=img_test(filename)

In [56]:
def mat_desc_train(img_train,descripteur):
    h=[]
    for i in range(42):
            h.append(descripteur(mat_img_train[i]))
    return np.array(h)

In [57]:
def mat_desc_test(img_test,descripteur):
    h=[]
    n=0
    for i in range(24):
        h.append(descripteur(mat_img_test[i]))
    return np.array(h)

# <span style="color: red;">Matrice du performance</span>

In [85]:
def dist_mat(desc_test,desc_train):
    L=np.zeros((24,42))
    f = np.zeros((24, 42), dtype=object)
    g=np.zeros((24,42))
    m=np.zeros((24,42))
    n=0
    for i in range(24):
        for j in range(42):
            L[i,j]=distance(desc_train[j],desc_test[i])
    for i in range(24):
        for j in range(42):
            f[i,j]=(L[i][j],j)
    for i in range(24):
        f[i]=sorted(f[i])
    for i in range(24):
        for j in range(42):
            g[i,j]=f[i][j][1]
    for i in range(24):
        if (i == 4 or i == 8 or i==12 or i == 16  or i==20 ):
            n=n+1
        for j in range(42):
            if 7*n<=g[i,j]<7*n+7:
                 m[i,j]=1
            else:
                m[i,j]=0
    return m

# <span style="color: red;">Matrice du précision et rappel(recall)</span>

In [92]:
def evalu(m):
    g=np.zeros((24,42))
    f=np.zeros((24,42))
    rec=np.zeros((1,42))
    prec=np.zeros((1,42))
    for i in range(24):  
        rec=np.zeros((1,42))
        prec=np.zeros((1,42))
        for j in range(1,43):
            TP=np.count_nonzero(m[i,0:j]==1)
            FP=np.count_nonzero(m[i,0:j]==0)
            FN=np.count_nonzero(m[i]==1)-TP
           #FN=20-TP
            prec[0,j-1]=TP/(TP+FP)
            rec[0,j-1]=TP/(TP+FN)
        g[i]=prec
        f[i]=rec
    return g,f

In [102]:
def moyen_prec(prec):
    s=0
    for i in range(24):
        s+=prec[i][0]
    return  s/24

# <span style="color: red;">Calcul du précision en utilisant l'histogramme</span>

In [86]:
mat_per_histo=dist_mat(mat_desc_test(mat_img_test,histo),mat_desc_train(mat_img_train,histo))

In [88]:
mat_per_histo

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [93]:
prec=evalu(mat_per_histo)[0]

In [94]:
prec

array([[1.        , 1.        , 0.66666667, ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 1.        , 0.66666667, ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 1.        , 1.        , ..., 0.175     , 0.17073171,
        0.16666667],
       ...,
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 1.        , 1.        , ..., 0.175     , 0.17073171,
        0.16666667]])

In [96]:
moyen_prec(prec)

0.5833333333333334

moyenne de precision en utilisant l'histogramme est : 0.5833333333333334

# <span style="color: red;">Calcul du précision en utilisant l'avrage_color</span>

In [98]:
mat_per_avrc=dist_mat(mat_desc_test(mat_img_test,avrage_color),mat_desc_train(mat_img_train,avrage_color))

In [99]:
mat_per_avrc

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [100]:
prec=evalu(mat_per_avrc)[0]

In [101]:
prec

array([[0.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 1.        , 1.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667],
       ...,
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667]])

In [104]:
moyen_prec(prec)

0.4583333333333333

moyenne de precision en utilisant l'avrage color est : 0.4583333333333333

# <span style="color: red;">Calcul du précision en utilisant texture</span>

In [106]:
mat_per_texture=dist_mat(mat_desc_test(mat_img_test,texture),mat_desc_train(mat_img_train,texture))

In [111]:
mat_per_texture

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [112]:
prec=evalu(mat_per_texture)[0]

In [113]:
prec

array([[0.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       ...,
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667]])

In [114]:
moyen_prec(prec)

0.2916666666666667

moyenne de precision en utilisant texture est : 0.2916666666666667

# <span style="color: red;">Calcul du précision en utilisant l'histogramme cumulé</span>

In [115]:
mat_per_histo_cum=dist_mat(mat_desc_test(mat_img_test,histo_cum),mat_desc_train(mat_img_train,histo_cum))

In [116]:
mat_per_histo_cum

array([[1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [117]:
prec=evalu(mat_per_histo_cum)[0]

In [118]:
prec

array([[1.        , 1.        , 0.66666667, ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 1.        , 1.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.5       , 0.66666667, ..., 0.175     , 0.17073171,
        0.16666667],
       ...,
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667]])

In [119]:
moyen_prec(prec)

0.7083333333333334

moyenne de precision en utilisant histogramme cumulé est : 0.7083333333333334

# <span style="color: red;">Calcul du précision en utilisant texture de la matrice de cooccurence</span>

In [194]:
mat_per_texture_cooccurence=dist_mat(mat_desc_test(mat_img_test,texture_cooccurence),mat_desc_train(mat_img_train,texture_cooccurence))

In [195]:
mat_per_texture_cooccurence

array([[1., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

In [196]:
prec=evalu(mat_per_texture_cooccurence)[0]

In [197]:
prec

array([[1.        , 0.5       , 0.66666667, ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.5       , 0.66666667, ..., 0.175     , 0.17073171,
        0.16666667],
       [1.        , 1.        , 1.        , ..., 0.175     , 0.17073171,
        0.16666667],
       ...,
       [1.        , 0.5       , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.        , ..., 0.175     , 0.17073171,
        0.16666667],
       [0.        , 0.        , 0.33333333, ..., 0.175     , 0.17073171,
        0.16666667]])

In [198]:
moyen_prec(prec)

0.5416666666666666

moyenne de precision en utilisant texture de la matrice du cooccurence est : 0.54

# <span style="color: red;">l'amilioration de la moyenne du precision</span>

## <span style="color: green;">Nouveau descripteur</span>

En utilisant les cinqs descripteurs précedents c'est à dire en fait une combinaison entre les cinqs discripteurs
pour créer un nouveau descripteur ,en effet en utilse dans chaque image le descripteur qui a un moyen de precession plus élevé 

In [199]:
def nouveau_mat_per(mat_per_histo,mat_per_histo_cum,mat_per_texture,mat_per_avrc,mat_per_texture_cooccurence):
    r=np.zeros((24,42))
    for i in range(24):
        for j in range(42):
            somme= mat_per_avrc[i,j]+mat_per_histo_cum[i,j]+mat_per_texture[i,j]+mat_per_histo[i,j]+mat_per_texture_cooccurence[i,j]
            if  somme!=0:
                 r[i,j]=1
    return r

In [200]:
nouveau_mat_per=nouveau_mat_per(mat_per_histo,mat_per_histo_cum,mat_per_texture,mat_per_avrc,mat_per_texture_cooccurence)

In [201]:
prec=evalu(nouveau_mat_per)[0]

In [202]:
prec

array([[1.        , 1.        , 1.        , ..., 0.5       , 0.48780488,
        0.47619048],
       [1.        , 1.        , 1.        , ..., 0.425     , 0.41463415,
        0.4047619 ],
       [1.        , 1.        , 1.        , ..., 0.4       , 0.3902439 ,
        0.38095238],
       ...,
       [1.        , 0.5       , 0.33333333, ..., 0.5       , 0.48780488,
        0.47619048],
       [1.        , 0.5       , 0.33333333, ..., 0.5       , 0.48780488,
        0.47619048],
       [1.        , 1.        , 1.        , ..., 0.625     , 0.6097561 ,
        0.5952381 ]])

In [203]:
moyen_prec(prec)*100

91.66666666666666